In [13]:
import os 
import cv2 
import matplotlib.pyplot as plt 
import time

In [2]:
from google.colab import drive

In [3]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [9]:
VIDEO_PATH = "/content/drive/MyDrive/dataset/image/John_Wick_small.mp4"

In [5]:
!mkdir ./pretrained
!wget -O ./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
!wget -O ./pretrained/config_graph.pbtxt https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt

--2022-06-28 02:24:20--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.11.128, 2607:f8b0:400c:c01::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.11.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381355771 (364M) [application/x-tar]
Saving to: ‘./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz’

./pretrained/faster 100%[===================>] 363.69M   283MB/s    in 1.3s    

2022-06-28 02:24:21 (283 MB/s) - ‘./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz’ saved [381355771/381355771]

--2022-06-28 02:24:22--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.git

In [6]:
!tar -xvf ./pretrained/faster*.tar.gz -C ./pretrained 

faster_rcnn_resnet50_coco_2018_01_28/
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.index
faster_rcnn_resnet50_coco_2018_01_28/checkpoint
faster_rcnn_resnet50_coco_2018_01_28/pipeline.config
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.meta
faster_rcnn_resnet50_coco_2018_01_28/saved_model/
faster_rcnn_resnet50_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_resnet50_coco_2018_01_28/saved_model/variables/
faster_rcnn_resnet50_coco_2018_01_28/frozen_inference_graph.pb


In [7]:
!pwd
!ls -lia ./pretrained/faster_rcnn_resnet50_coco_2018_01_28

/content
total 296080
2490642 drwxr-xr-x 3 345018 5000      4096 Feb  1  2018 .
2490638 drwxr-xr-x 3 root   root      4096 Jun 28 02:24 ..
2490644 -rw-r--r-- 1 345018 5000        77 Feb  1  2018 checkpoint
2490651 -rw-r--r-- 1 345018 5000 120549957 Feb  1  2018 frozen_inference_graph.pb
2490646 -rw-r--r-- 1 345018 5000 176914228 Feb  1  2018 model.ckpt.data-00000-of-00001
2490643 -rw-r--r-- 1 345018 5000     14460 Feb  1  2018 model.ckpt.index
2490647 -rw-r--r-- 1 345018 5000   5675175 Feb  1  2018 model.ckpt.meta
2490645 -rw-r--r-- 1 345018 5000      3240 Feb  1  2018 pipeline.config
2490648 drwxr-xr-x 3 345018 5000      4096 Feb  1  2018 saved_model


In [8]:
cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/faster_rcnn_resnet50_coco_2018_01_28/frozen_inference_graph.pb', 
                                     './pretrained/config_graph.pbtxt')

In [19]:
# OpenCV Yolo
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorbike',4:'aeroplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',10:'fire hydrant',
                        11:'stop sign',12:'parking meter',13:'bench',14:'bird',15:'cat',16:'dog',17:'horse',18:'sheep',19:'cow',20:'elephant',
                        21:'bear',22:'zebra',23:'giraffe',24:'backpack',25:'umbrella',26:'handbag',27:'tie',28:'suitcase',29:'frisbee',30:'skis',
                        31:'snowboard',32:'sports ball',33:'kite',34:'baseball bat',35:'baseball glove',36:'skateboard',37:'surfboard',38:'tennis racket',39:'bottle',40:'wine glass',
                        41:'cup',42:'fork',43:'knife',44:'spoon',45:'bowl',46:'banana',47:'apple',48:'sandwich',49:'orange',50:'broccoli',
                        51:'carrot',52:'hot dog',53:'pizza',54:'donut',55:'cake',56:'chair',57:'sofa',58:'pottedplant',59:'bed',60:'diningtable',
                        61:'toilet',62:'tvmonitor',63:'laptop',64:'mouse',65:'remote',66:'keyboard',67:'cell phone',68:'microwave',69:'oven',70:'toaster',
                        71:'sink',72:'refrigerator',73:'book',74:'clock',75:'vase',76:'scissors',77:'teddy bear',78:'hair drier',79:'toothbrush' }

In [20]:
# OpenCV Tensorflow Faster-RCNN
labels_to_names_0 = {0:'person',1:'bicycle',2:'car',3:'motorcycle',4:'airplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',
                    10:'fire hydrant',11:'street sign',12:'stop sign',13:'parking meter',14:'bench',15:'bird',16:'cat',17:'dog',18:'horse',19:'sheep',
                    20:'cow',21:'elephant',22:'bear',23:'zebra',24:'giraffe',25:'hat',26:'backpack',27:'umbrella',28:'shoe',29:'eye glasses',
                    30:'handbag',31:'tie',32:'suitcase',33:'frisbee',34:'skis',35:'snowboard',36:'sports ball',37:'kite',38:'baseball bat',39:'baseball glove',
                    40:'skateboard',41:'surfboard',42:'tennis racket',43:'bottle',44:'plate',45:'wine glass',46:'cup',47:'fork',48:'knife',49:'spoon',
                    50:'bowl',51:'banana',52:'apple',53:'sandwich',54:'orange',55:'broccoli',56:'carrot',57:'hot dog',58:'pizza',59:'donut',
                    60:'cake',61:'chair',62:'couch',63:'potted plant',64:'bed',65:'mirror',66:'dining table',67:'window',68:'desk',69:'toilet',
                    70:'door',71:'tv',72:'laptop',73:'mouse',74:'remote',75:'keyboard',76:'cell phone',77:'microwave',78:'oven',79:'toaster',
                    80:'sink',81:'refrigerator',82:'blender',83:'book',84:'clock',85:'vase',86:'scissors',87:'teddy bear',88:'hair drier',89:'toothbrush',
                    90:'hair brush'}

In [21]:
labels_to_names = {1:'person',2:'bicycle',3:'car',4:'motorcycle',5:'airplane',6:'bus',7:'train',8:'truck',9:'boat',10:'traffic light',
                    11:'fire hydrant',12:'street sign',13:'stop sign',14:'parking meter',15:'bench',16:'bird',17:'cat',18:'dog',19:'horse',20:'sheep',
                    21:'cow',22:'elephant',23:'bear',24:'zebra',25:'giraffe',26:'hat',27:'backpack',28:'umbrella',29:'shoe',30:'eye glasses',
                    31:'handbag',32:'tie',33:'suitcase',34:'frisbee',35:'skis',36:'snowboard',37:'sports ball',38:'kite',39:'baseball bat',40:'baseball glove',
                    41:'skateboard',42:'surfboard',43:'tennis racket',44:'bottle',45:'plate',46:'wine glass',47:'cup',48:'fork',49:'knife',50:'spoon',
                    51:'bowl',52:'banana',53:'apple',54:'sandwich',55:'orange',56:'broccoli',57:'carrot',58:'hot dog',59:'pizza',60:'donut',
                    61:'cake',62:'chair',63:'couch',64:'potted plant',65:'bed',66:'mirror',67:'dining table',68:'window',69:'desk',70:'toilet',
                    71:'door',72:'tv',73:'laptop',74:'mouse',75:'remote',76:'keyboard',77:'cell phone',78:'microwave',79:'oven',80:'toaster',
                    81:'sink',82:'refrigerator',83:'blender',84:'book',85:'clock',86:'vase',87:'scissors',88:'teddy bear',89:'hair drier',90:'toothbrush',
                    91:'hair brush'}


In [22]:
cap = cv2.VideoCapture(VIDEO_PATH)
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"total frame count {frame_cnt}")

total frame count 58


In [23]:
VIDEO_OUT_PATH = "./data/John_Wick_small_cv01.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)

codec = cv2.VideoWriter_fourcc(*"XVID")

video_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video_fps = cap.get(cv2.CAP_PROP_FPS)

video_writer = cv2.VideoWriter(VIDEO_OUT_PATH, codec, video_fps, video_size)

frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"total frame count {frame_cnt}")

total frame count 58


In [12]:
GREEN_COLOR = (0, 255, 0)
RED_COLOR = (0, 0, 255)

In [48]:
def get_detected_image(image_array, score_threshold, model_name = cv_net, use_copied_array = True, is_print= True):
    rows = image_array.shape[0]
    cols = image_array.shape[1]

    draw_image = None
    if use_copied_array:
        draw_image = image_array.copy()
    else: 
        draw_image = image_array

    cv_net.setInput(cv2.dnn.blobFromImage(image_array, swapRB=True, crop=False))
    start = time.time()
    cv_out = cv_net.forward()

    green_color = (0, 255, 0)
    red_color = (0, 0, 255)

    for detection in cv_out[0,0, : ,: ]:
        score = float(detection[2])
        class_id = int(detection[1])

        if score >  score_threshold:
            left = detection[3] * cols 
            top = detection[4] * rows
            right = detection[5] * cols 
            bottom = detection[6] * rows

            caption = "{} : {:.4f}".format(labels_to_names_0[class_id], score)

            print(caption)

            cv2.rectangle(draw_image, (int(left), int(top)), (int(right), int(bottom)), color=green_color, thickness=2)
            cv2.putText(draw_image, caption, (int(left), int(top -5 )), cv2.FONT_HERSHEY_SIMPLEX, 0.4, red_color , 1)
    if is_print:
        print(f"detection time {round(time.time() - start, 2)}") 

    return draw_image 

In [40]:
while True:
    has_frame, image_frame = cap.read()

    if not has_frame:
        print("No more frames to process")
        break
    
    rows = image_frame.shape[0]
    cols = image_frame.shape[1]

    cv_net.setInput(cv2.dnn.blobFromImage(image_frame, swapRB=True, crop=False))

    start = time.time()

    cv_out= cv_net.forward()
    frame_index = 0 

    for detection in cv_out[0, 0, :, :]:
        score = float(detection[2])
        class_id = int(detection[1])

        if score > 0.5:

            left = detection[3] * cols 
            top = detection[4] * rows
            right = detection[5] * cols 
            bottom = detection[6] * rows
            caption = "{}: {:.4f}".format(labels_to_names_0[class_id], score)

            cv2.rectangle(image_frame, (int(left), int(top)), (int(right), int(bottom)), color=GREEN_COLOR, thickness=2)
            cv2.putText(image_frame, caption , (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, RED_COLOR, 1)
    
    print(f"detection time : {round(time.time() - start, 2)}")
    video_writer.write(image_frame)

video_writer.release()
cap.release()


No more frames to process


In [51]:
def do_detected_video(cv_net, input_path, output_path, score_threshold, is_print):
    
    cap = cv2.VideoCapture(VIDEO_PATH)

    codec = cv2.VideoWriter_fourcc(*'XVID')

    video_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    print(vid_size)
    vid_writer = cv2.VideoWriter(output_path, codec, video_fps, video_size) 

    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('total Frame count:', frame_cnt)

    green_color=(0, 255, 0)
    red_color=(0, 0, 255)
    while True:
        has_Frame, img_frame = cap.read()
        if not has_Frame:
            print('No more frames to process.')
            break
        
        img_frame = get_detected_image(img_frame, model_name =cv_net, score_threshold=score_threshold, use_copied_array=False, is_print=is_print)
                                                                      
        
        vid_writer.write(img_frame)


    vid_writer.release()
    cap.release()

In [50]:
do_detected_video(cv_net, '/content/drive/MyDrive/dataset/image/Jonh_Wick_small.mp4', '/content/John_Wick_small_02.mp4', 0.2, False)

(1072, 460)
total Frame count: 58
person : 0.9495
person : 0.2871
bicycle : 0.3498
car : 0.9882
car : 0.9622
car : 0.9266
car : 0.8772
car : 0.3988
car : 0.3763
car : 0.2583
car : 0.2150
motorcycle : 0.6607
book : 0.2069
person : 0.9581
person : 0.4459
person : 0.2940
bicycle : 0.4216
car : 0.9881
car : 0.9643
car : 0.9318
car : 0.8992
car : 0.4418
car : 0.4189
car : 0.2545
car : 0.2318
motorcycle : 0.5682
person : 0.7814
person : 0.5169
person : 0.3087
car : 0.9903
car : 0.9833
car : 0.9115
car : 0.8625
car : 0.4893
car : 0.4251
motorcycle : 0.9532
airplane : 0.2046
person : 0.7585
person : 0.7484
person : 0.7375
person : 0.7200
person : 0.3580
person : 0.2285
person : 0.2171
car : 0.9919
car : 0.9909
car : 0.9870
car : 0.7361
car : 0.6120
motorcycle : 0.8655
umbrella : 0.4362
person : 0.5313
person : 0.4869
person : 0.4709
car : 0.9933
car : 0.9817
car : 0.9530
car : 0.7823
car : 0.2064
motorcycle : 0.9781
truck : 0.4717
person : 0.8299
person : 0.4792
person : 0.2719
car : 0.9799
ca